###  Mount Google Drive

In [1]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3


Mounted at /content/drive


### Clone Codebase

In [2]:
! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3
! cd /content/drive/MyDrive/COMP4901B-Homework3 && git clone https://github.com/hkust-nlp/COMP4901B-LLMs.git

Cloning into 'COMP4901B-LLMs'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 232 (delta 40), reused 24 (delta 24), pack-reused 169 (from 1)
Receiving objects: 100% (232/232), 798.55 KiB | 1.87 MiB/s, done.
Resolving deltas: 100% (87/87), done.


### Download Dataset

In [3]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/setup.sh

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 83.1 MB/s eta 0:00:00

### Q1 code evaulation for q1

In [4]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3/ && python tests/test_format_prompts.py


SANITY CHECK: format_prompts()

This script tests your implementation of the format_prompts function.
It compares your outputs with expected reference outputs.


TEST 2: Few-shot without chat template
2025-11-13 09:43:54.067356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763027034.487824    6130 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763027034.594252    6130 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763027035.283583    6130 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763027035.283628    6130 computation_placer.cc:

### Q3 Start The Tuning

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3/ && bash scripts/self_train_gsm8k.sh